### OD-3. Принятие решения по результатам A/Б-теста

#### Рассчитаем насколько различаются конверсия, средний чек и ARPU пользователей в группах А и Б (ID теста 127, даты проведения: с 2019-08-05 до 2019-08-11 включительно).

### Часть 1. Подготовка датафреймов.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.weightstats import ztest
from datetime import datetime

In [2]:
ab_test_groups = pd.read_csv('ab_test_groups.csv')
ab_test_groups.head()

,user_id,grp,ab_test_id
0,1,B,132
1,2,A,132
2,3,B,132
3,4,A,132
4,5,B,132


In [3]:
ab_test_groups_2 = ab_test_groups.loc[ab_test_groups.ab_test_id == 127]

In [4]:
payments = pd.read_csv('payments.csv')
payments.head()

,payment_id,user_id,price,created_at,tariff
0,1,77831,200.0,2019-08-05 00:00:54,month
1,2,84576,200.0,2019-08-05 00:01:05,month
2,3,40981,140.0,2019-08-05 00:03:47,month_30off
3,4,130565,140.0,2019-08-05 00:09:33,month_30off
4,5,34758,700.0,2019-08-05 00:12:16,year_30off


In [5]:
payments_2 = payments.loc[(payments.created_at.astype(str) >= '2019-08-05')&
            (payments.created_at.astype(str) <= '2019-08-11'), ['user_id', 'price']]
payments_2.head()

,user_id,price
0,77831,200.0
1,84576,200.0
2,40981,140.0
3,130565,140.0
4,34758,700.0


In [6]:
payments_2 = payments_2.groupby('user_id', as_index = False).agg({'price':'sum'})

In [7]:
ab_data_2 = ab_test_groups_2.merge(payments_2, on = 'user_id', how = 'left')
print(ab_data_2.head())
print(ab_data_2.info())

   user_id grp  ab_test_id  price
0        1   B         127  140.0
1        2   B         127    NaN
2        3   B         127    NaN
3        4   B         127    NaN
4        5   A         127    NaN
<class 'pandas.core.frame.DataFrame'>
Int64Index: 153232 entries, 0 to 153231
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     153232 non-null  int64  
 1   grp         153232 non-null  object 
 2   ab_test_id  153232 non-null  int64  
 3   price       12027 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 5.8+ MB
None


In [8]:
ab_summary_2 = ab_data_2.groupby('grp').agg({'user_id':'count', 'price':'count'}) #Агрегация данных
ab_summary_2.rename(columns={'user_id':'users_count', 'price':'buyers_count'}, inplace = True) #Переименовывание столбцов
ab_summary_2

,users_count,buyers_count
grp,,
A,76605,3723
B,76627,8304


### Часть 2. Подсчет конверсии, среднего чека и ARPU.

In [9]:
ab_summary_2['conversion'] = ab_summary_2.buyers_count/ab_summary_2.users_count
ab_summary_2

,users_count,buyers_count,conversion
grp,,,
A,76605,3723,0.048600
B,76627,8304,0.108369


In [10]:
ab_summary_2 = ab_summary_2.merge(ab_data_2.groupby('grp').agg({'price':'mean'}).rename(columns = {'price':'avg_bill'}), 
                 left_index = True, right_index = True) #Сделали агрегацию и прикрепили её к ab_summary
ab_summary_2

,users_count,buyers_count,conversion,avg_bill
grp,,,,
A,76605,3723,0.048600,398.334676
B,76627,8304,0.108369,350.067437


In [11]:
ab_summary_2 = ab_summary_2.merge(ab_data_2.groupby('grp').agg({'price':'sum'}).rename(columns = {'price':'revenue'}), 
                 left_index = True, right_index = True) # Сделали агрегацию и прикрепили её к ab_summary
ab_summary_2['ARPU'] = ab_summary_2.revenue/ab_summary_2.users_count
ab_summary_2['Доп. выручка'] = ab_summary_2.users_count*(ab_summary_2.ARPU - ab_summary_2.loc['A', 'ARPU'])/100
ab_summary_2

,users_count,buyers_count,conversion,avg_bill,revenue,ARPU,Доп. выручка
grp,,,,,,,
A,76605,3723,0.048600,398.334676,1483000.0,19.359050,0.000000
B,76627,8304,0.108369,350.067437,2906960.0,37.936498,14235.341009


In [12]:
print('Конверсия в группе А: ', round(ab_summary_2['conversion'][0], 3))
print('Конверсия в группе B: ', round(ab_summary_2['conversion'][1], 3))

print('Средний чек в группе А: ', round(ab_summary_2['avg_bill'][0], 2))
print('Средний чек в группе B: ', round(ab_summary_2['avg_bill'][1], 2))

print('ARPU в группе А: ', round(ab_summary_2['ARPU'][0], 2))
print('ARPU в группе B: ', round(ab_summary_2['ARPU'][1], 2))

print('Доп. выручка в группе B: ', round(ab_summary_2['Доп. выручка'][1], 1))
ab_summary_2['ARPU'][1]-ab_summary_2['ARPU'][0]

Конверсия в группе А:  0.049
Конверсия в группе B:  0.108
Средний чек в группе А:  398.33
Средний чек в группе B:  350.07
ARPU в группе А:  19.36
ARPU в группе B:  37.94
Доп. выручка в группе B:  14235.3


18.57744790879522

### Часть 3. Выводы.

#### Исходя из расчетов видно, что выручка в группе В выше более чем в два раза чем в группе А. Средний чек снизился примерно на 13 процентов, однако, ARPU выросла почти в два раза. Это в итоге привело к получению дополнительной выручки в размере 14235.3.
#### Таким образом, можно сделать вывод, что фича успешна и способствует повышению прибыли.